# Conexión Mongo-Redis-PANDAS

## Instalar dependencias

In [ ]:
!conda install -y -c conda-forge pymongo redis-py

### Importar librerías

In [ ]:
import redis
from pymongo import MongoClient
import pandas as pd

In [ ]:
def conexion_mongo():
    HOST="XXX"
    PORT=27017
    USERNAME="XXX"
    PASSWORD="XXX"
    cli_mongo = MongoClient(f"mongodb+srv://{USERNAME}:{PASSWORD}@{HOST}/test")
    return cli_mongo

def conexion_redis():
    redis_cli = redis.Redis()
    #redis_cli.auth("123quetal123")
    #redis_cli.ping()
    return redis_cli

#Conexión Mongo
cli_mongo=conexion_mongo()
# @ sample_guides -> planets
db_guides=cli_mongo['sample_guides']
coleccion_planetas=db_guides['planets']

#Conexión Redis
redis_cli = conexion_redis()
redis_cli.flushall()

aux_documentos = coleccion_planetas.find({})
for documento in aux_documentos:
    idPlaneta = documento.pop('_id')
    documento.update({"_id": str(idPlaneta) })
    redis_cli.json().set(documento.get('name'), "$", documento)

listaPlanetas = []

for i in redis_cli.scan_iter():
    auxPlaneta = redis_cli.json().get(i.decode(), '$')
    listaPlanetas.append(auxPlaneta[0])

df = pd.DataFrame(listaPlanetas)

df

## De Mongo DB a PANDAS

In [ ]:
dfPlanetasDirectos = pd.DataFrame.from_records(coleccion_planetas.find())
dfPlanetasDirectos